In [1]:
import numpy as np
from pyomo.environ import *

# Step 1 - DC load flow using PTDF in Python

In [2]:
# from IPython.display import Image
# Image(filename="C:/Users/fmansoor/Downloads/4bus/4bus_grid.png",width=500,height=500)

In [3]:
f_old = np.array([20,180,200,240,-20])
print(f_old)

[ 20 180 200 240 -20]


In [1]:
M=np.array([[0.4,0.2,0.3,0.35],[0.6,0.4,-0.3,-0.35],[-0.2,-0.6,-0.4,-0.3],[-0.4,-0.2,-0.3,-0.35],[-0.2,0.4,0.6,-0.3]])
print(M)

NameError: name 'np' is not defined

In [37]:
M[:,:2]

array([[ 0.4 ,  0.  ,  0.3 ,  0.35],
       [ 0.6 ,  0.  , -0.3 , -0.35],
       [-0.2 ,  0.  , -0.4 , -0.3 ],
       [-0.4 ,  0.  , -0.3 , -0.35],
       [-0.2 ,  0.  ,  0.6 , -0.3 ]])

In [38]:
dP = np.array([-37.5,-62.5])
print(dP)

[-37.5 -62.5   0.    0. ]


In [39]:
df=M[:,:2]*dP
print(df)

[[-15.   -0.    0.    0. ]
 [-22.5  -0.   -0.   -0. ]
 [  7.5  -0.   -0.   -0. ]
 [ 15.   -0.   -0.   -0. ]
 [  7.5  -0.    0.   -0. ]]


In [40]:
(M[:,:2]*dP).sum(axis=1)

array([-15. , -22.5,   7.5,  15. ,   7.5])

In [41]:
f_new = (M[:,:2]*dP).sum(axis=1) + f_old
print(f_new)

[  5.  157.5 207.5 255.  -12.5]


# Step 2 - Optimization with Python

In [10]:
model = ConcreteModel()

In [11]:
c = np.array([100,100]) #cost of generation
P_old = np.array([50,400]) #old generation at generation buses
Load = np.array([650,350]) #load 
P_max = np.array([1000,1000]) #max. generation at bus
f_max = np.array([300,300,300,270,300]) #line thermal limit
f_old = np.array([20,180,200,240,-20]) #base line flows

In [13]:
# model.del_component(model.load_const)

In [14]:
index = range(2)
print(index)

range(0, 2)


In [15]:
# def P_old1(model,i):
#     return P_old[i]
# model.P_old1 = Param(index,initialize=P_old1)

In [16]:
model.P_new = Var(index,initialize=0) #new generation at generation bus

In [17]:
def obj_rule(model):
    obj = sum(c[i]*(model.P_new[i]+P_old[i]) for i in index)
    print(obj)
    return obj
model.z = Objective(rule=obj_rule,sense=minimize)

100*(P_new[0] + 50) + 100*(P_new[1] + 400)


In [18]:
def max_gen_rule(model,i):
    max_gen = model.P_new[i] + P_old[i] <= P_max[i]
    print(max_gen)
    return max_gen
model.Gen_const = Constraint(index,rule=max_gen_rule)

P_new[0] + 50  <=  1000.0
P_new[1] + 400  <=  1000.0


In [19]:
def load_rule(model):
    ld = sum(model.P_new[i] for i in index)+sum(P_old) == sum(Load)
    print(ld)
    return ld
model.load_const = Constraint(rule=load_rule)

P_new[0] + P_new[1] + 450  ==  1000.0


In [20]:
# for n in index:
#     dP[n] = model.P_old1[n] - model.P_new[n]
# print(dP)

In [21]:
k=range(5)

In [22]:
def line_rule(model,i,j):
    val = (M[j,i]*(P_old[i] - model.P_new[i])) + f_old[j]/len(index)
    print(val)
    return (-f_max[j]/len(index), val, f_max[j]/len(index))
model.Line_const = Constraint(index, k,rule=line_rule)

0.4*(50 - P_new[0]) + 10.0
0.6*(50 - P_new[0]) + 90.0
-0.2*(50 - P_new[0]) + 100.0
-0.4*(50 - P_new[0]) + 120.0
-0.2*(50 - P_new[0]) - 10.0
0.2*(400 - P_new[1]) + 10.0
0.4*(400 - P_new[1]) + 90.0
-0.6*(400 - P_new[1]) + 100.0
-0.2*(400 - P_new[1]) + 120.0
0.4*(400 - P_new[1]) - 10.0


In [23]:
# def line_rule(model,i,j):
#     val = (model.val1) + f_old[j]
#     print(val)
#     return (-f_max[j], val, f_max[j])
# model.Line_const = Constraint(index, k,rule=line_rule)

In [24]:
# model.val1=Var(index,initialize=0,within=Any)

In [25]:
# for n in index:
#     model.val1[n] = (M[:,n]*(P_old[n] - model.P_new[n]))

In [26]:
# model.val1

In [27]:
# model.del_component(model.Li)

In [28]:
SolverFactory('glpk').solve(model).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 100000.0
  Upper bound: 100000.0
  Number of objectives: 1
  Number of constraints: 24
  Number of variables: 3
  Number of nonzeros: 25
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.06897091865539551
# ----------------------------------------------------------
#   Solution Information
# -----------------------------

In [29]:
print(model.P_new[0]())
print(model.P_new[1]())

87.5
462.5


In [30]:
print((M[:,0]*(P_old[0] - model.P_new[0]())) + (M[:,1]*(P_old[1] - model.P_new[1]())) + f_old)

[ -7.5 132.5 245.  267.5 -37.5]


In [31]:
f_old

array([ 20, 180, 200, 240, -20])